In [3]:
import pyvista as pv
import numpy as np
from pathlib import Path

In [4]:
avatar_path = Path("../").resolve() / "data" / "spread_arms.obj"
avatar_mesh = pv.read(avatar_path)

avatar_mesh = avatar_mesh.scale([0.01, 0.01, 0.01])

/Users/vikramsingh/Library/Caches/pypoetry/virtualenvs/pbd-GYzgUBp8-py3.8/lib/python3.8/site-packages/pyvista/core/pointset.py:262: PyVistaDeprecationWarning: You did not specify a value for `inplace` and the default value will be changing to `False` in future versions for point-based meshes (e.g., `PolyData`). Please make sure you are not assuming this to be an inplace operation.
  warnings.warn(DEFAULT_INPLACE_WARNING, PyVistaDeprecationWarning)


In [5]:
lower_right_points = [[0, 0.95, 0.25], [0.25, 0.95, 0.25], [0.25, 1.25, 0.25]]
right_sleeve_points = [[0.35, 1.20, 0.25], [0.35, 1.39,0.25] ]
center_points = [[0.15, 1.55,0.25], [-0.15, 1.55,0.25],]
left_sleeve_points = [[-0.35, 1.39,0.25], [-0.35, 1.20, 0.25]]
lower_left_points = [[-0.25, 1.25, 0.25], [-0.25, 0.95, 0.25], [0., 0.95, 0.25]]

tshirt_points = lower_right_points + right_sleeve_points + center_points + left_sleeve_points + lower_left_points
tshirt_lines = pv.lines_from_points(tshirt_points)

In [9]:
def create_mesh(closed_lines: pv.PolyData) -> pv.PolyData:
    with pygmsh.geo.Geometry() as geom:
        geom.add_polygon(
            closed_lines,
            mesh_size=1.0,
        )

        mesh: meshio.Mesh = geom.generate_mesh(
            dim=2, algorithm=5, verbose=True
        )  # 5: Delaunay

    # HardCoding alert! We observe that [1] has triangle elements so we use that
    # https://github.com/meshpro/pygmsh/discussions/524
    elems = mesh.cells[1].data
    coords = mesh.points

    cells = [("triangle", elems)]
    meshio.write_points_cells("/tmp/out.vtk", coords, cells)

    mesh_pv = pv.read("/tmp/out.vtk")

    return mesh_pv

triangle_mesh = create_mesh(tshirt_lines)

NameError: name 'pygmsh' is not defined

In [49]:
center = (0, 1.1, 0.25)

plane = pv.Plane(center=center, direction=(0, 0, 1), i_size=0.5, j_size=0.3)

plotter = pv.Plotter()
plotter.add_mesh(avatar_mesh, color="yellow")
# plotter.add_mesh(plane, color="blue")
plotter.add_mesh(tshirt_triangles, color = "blue", show_edges=True)

# plotter.add_point_labels(plane.points[100], labels = [f"{str(plane.points[100])}"])

plotter.add_axes()
plotter.show(cpos = 'xy')

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)